In [1]:
from faster_whisper import WhisperModel
import os

C:\Users\esaya\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
video_path = "../examples/pilots_audio.mp3"

In [8]:
class STT:
    def __init__(self, model_size="base", device="cpu"):
        """
        Initialize the STT class with the specified model size and device.

        Parameters:
        model_size (str): The size of the Whisper model to use (e.g., "tiny", "base", "small", "medium", "large").
                            for english only: "tiny.en", "base.en", "small.en", "medium.en" are available and perform better.
        device (str): The device to run the model on ("cpu" or "cuda").
        """
        self.model_size = model_size
        self.device = device
        self.model = None
        self._load_model()

    def _load_model(self):
        """
        Load the Whisper model. The model will be downloaded if it is not already cached locally.
        """
        self.model = WhisperModel(self.model_size, device=self.device)

    def format_time(self, seconds):
        minutes, seconds = divmod(seconds, 60)
        hours, minutes = divmod(minutes, 60)
        milliseconds = (seconds - int(seconds)) * 1000
        return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d},{int(milliseconds):03d}"

    def transcribe(self, input_file):
        task = "transcribe"  # translate, transcribe
        segments, info = self.model.transcribe(input_file, vad_filter=True, beam_size=5)
        print("Detected language '{}' with probability {:.2f}".format(info.language, info.language_probability))

        return segments

    def get_srt(self, segments, input_file):
        srt_filename = os.path.splitext(input_file)[0] + '.srt'
        try:
            with open(srt_filename, 'w', encoding='utf-8') as srt_file:
                for segment in segments:
                    start_time = self.format_time(segment.start)
                    end_time = self.format_time(segment.end)
                    text = segment.text
                    segment_id = segment.id
                    line_out = f"{segment_id}\n{start_time} --> {end_time}\n{text.lstrip()}\n\n"
                    srt_file.write(line_out)
        except Exception as e:
            print(f"Error writing srt file: {e}")

        return srt_filename


In [13]:
stt = STT(model_size="base.en", device="cpu")

c:\Users\esaya\.conda\envs\microserv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\esaya\.cache\huggingface\hub\models--Systran--faster-whisper-base.en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [14]:
segments = stt.transcribe(input_file=video_path)

Detected language 'en' with probability 1.00


In [15]:
srt_path = stt.get_srt(segments, video_path)

In [12]:
srt_path

'../examples/pilots_audio.srt'